# Active Account Turnover Tracking MVP

This is just an example of using widgets for UI, python/sql for logic, and sqlite3 for local data storage.

In [1]:
from IPython.display import display
import ipywidgets as widgets
from datetime import date
start_date_input = widgets.DatePicker(description="start date",value=date(2018,4,13))
end_date_input = widgets.DatePicker(description="end date", value=date(2018,4,15))

trader_code_dropdown = widgets.Dropdown(description="Trader Code",options=['dcooch', '2', '3'])

display(start_date_input)
display(end_date_input)
display(trader_code_dropdown)

DatePicker(value=datetime.date(2018, 4, 13), description='start date')

DatePicker(value=datetime.date(2018, 4, 15), description='end date')

Dropdown(description='Trader Code', options=('dcooch', '2', '3'), value='dcooch')

## User Interface 

The user interface is generated by Widgets, which are an extension of the Jupyter labs notebook environment. These cells are design to be run one at a time, with the input data filled out by the user as they go. One the above cell is run, two date pickers and a dropdown will appear. The user selects the values they want, and then continues by running the next cell.

In [225]:
import pyodbc
import pandas as pd
conn = pyodbc.connect('DSN=dev-sma')
cursor = conn.cursor()

start_date = start_date_input.value
end_date = end_date_input.value
trader_code = trader_code_dropdown.value

sql = """SELECT [SMA].[sma].[portfolio].portfolio_id,
latest_transaction.cusip,
[SMA].[sma].[portfolio].port_mgmt_style,
[SMA].[muni].[transaction_allocation].transaction_id,
latest_transaction.transaction_date, latest_transaction.transaction_price, sma.muni.transaction_allocation.amount,
latest_transaction.trader_code
  FROM [SMA].[sma].[portfolio]
  INNER JOIN [SMA].[muni].[transaction_allocation]
  ON [SMA].[sma].[portfolio].portfolio_id = [SMA].[muni].[transaction_allocation].portfolio_id
  INNER JOIN (SELECT transaction_date, transaction_id, trader_code, transaction_price, cusip
      FROM [SMA].[muni].[transaction] WHERE transaction_date BETWEEN '{0}' and '{1}' and trader_code = '{2}' and transaction_price is not NULL) as latest_transaction
  ON [SMA].[muni].[transaction_allocation].transaction_id = latest_transaction.transaction_id
  WHERE port_mgmt_style ='Intermediate (SMA 6 yr dur)'  or port_mgmt_style = 'Limited' or port_mgmt_style = 'Long'
 ORDER BY transaction_date ASC""".format(start_date.strftime("%Y/%m/%d"), end_date.strftime("%Y/%m/%d"), trader_code)

transaction_data = pd.read_sql(sql,conn)


## Fetch Trades 
This selects the trades for the trader given the input date above. These are then stored directly from SQL to a dataframe called transaction_data.

In [226]:
transaction_data['transaction_date'] = transaction_data['transaction_date'].map(lambda val: val.strftime("%Y/%m/%d"))

#### Cleaning
This cleans the data. The transaction date in the pandas data frame comes through as a datetime object, and therefore is not serializable for the ipysheet package. So here I convert it to a string date. 

In [227]:
transaction_data.insert(0, "tags",None)

#### Adding input field 
This adds an input field onto the data frame called `tags`. This will allow users to input common delimiated text to tag various position.

In [228]:
from ipysheet import sheet, from_dataframe, to_dataframe

# create sheet
sheet = from_dataframe(transaction_data)
# display sheet
sheet


Sheet(cells=(Cell(column_end=0, column_start=0, numeric_format=None, row_end=30, row_start=0, squeeze_row=Fals…

In [229]:
transaction_data = to_dataframe(sheet)

#### Save edited data

When the user is done tagging certain trades, the `to_dataframe` function saves the sheet back as a data frame which will be uploaded to a local .db file.

In [230]:
today = date.today().strftime("%Y/%m/%d")
values_to_save = transaction_data[transaction_data.tags.notnull()]
values_to_save.insert(0, "effective_date",today)
values_to_database = values_to_save[["cusip","portfolio_id", "amount","tags","effective_date", "trader_code"]]

#### What to save 

Certain columns as selected to be saved for each tagged position

In [231]:
import sqlite3
con = sqlite3.connect('positions_selected_for_turnover.db')
cur = con.cursor()

result = cur.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='position_tags'").fetchall()
if len(result) == 0:
    print("position_tags table does not exist, creating it now.")
    # Create table
    cur.execute('''CREATE TABLE IF NOT EXISTS position_tags
                (cusip, portfolio_id, amount, tags, effective_date, trader_code)''')
    # Save (commit) the changes
    con.commit()
con.close()



#### Create our Table 

This will only created the database file is it does not exist yet. Otherwise this cell does nothing. 

In [232]:
con = sqlite3.connect('positions_selected_for_turnover.db')

values_to_database.to_sql('position_tags', con=con, if_exists="append", index=False)

con.close()

#### Save The Data

This saves the selected data to the position_tags table inside the `positions_selected_for_turnover` database.


In [233]:
con = sqlite3.connect('positions_selected_for_turnover.db')
cur = con.cursor()
sql = "select * from position_tags"
stored_positon_tags = pd.read_sql(sql,con)

position_sheet = from_dataframe(stored_positon_tags)
position_sheet

Sheet(cells=(Cell(column_end=0, column_start=0, numeric_format=None, row_start=0, squeeze_row=False, type='tex…

### Confirm Results 

Double check that we did indeed store some values in the data base

In [234]:
stored_positon_tags.to_excel("stored_position_tags.xlsx")

### Export to Excell

Example of how easy it is to export from the database to an excell file.